### Автоматизация отчетности

Продуктовая команда попросила настроить ежедневную рассылку в телеграм с отчетом по ленте новостей. Отчет должен состоять из двух частей:
- текст с информацией о значениях ключевых метрик за предыдущий день
- график с отображением динамики DAU за предыдущие 7 дней

В отчете должны быть отражены следующие ключевые метрики: 
- DAU 
- Просмотры
- Лайки
- CTR

Отчет должен приходить ежедневно в 11.00

Для реализации поставленной задачи напишем DAG, который подгрузим в Apache Airflow. Данная программа позволит ежедневно в заданное время обрабатывать скрипт и запускать рассылку с использованием телеграм-бота.

In [ ]:
#импортируем необходимые библиотеки
import requests
import json
from urllib.parse import urlencode
import pandahouse
import seaborn as sns 
import io
import matplotlib.pyplot as plt
from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
from airflow.models import Variable
from datetime import timedelta
from datetime import datetime

In [55]:
#зададим параметры DAG
    default_args = {
    'owner': 'a-lelkova',
    'depends_on_past': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2024, 8, 27)   
}

Перед написанием скрипта в телеграм создадим бота, дадим ему имя и уникальный никнейм, запустим его и скопируем token.

Чтобы узнать id необходимого чата, в строку поиска браузера вставим https://api.telegram.org/bot{token}/getUpdates и подставим token нашего бота.

В открывшемся окне будет содержание json файла, где будет содержаться id чата (result > 0 > chat > id).

In [56]:
#напишем скрипт для подсчета и визуализации необходимых данных и оформим его в DAG
@dag('lelkova_report_DAG',default_args=default_args, schedule_interval='0 11 * * *', catchup=False)
def lelkova_report_DAG():
    #все расчеты будут производится в рамках одного таска 
    @task(retries=3)
    #зададим функцию, которая будет собирать наш отчет
    def report(chat = None):
        # укажем id чата, в который нужно будет направлять рассылку
        chat_id =  chat or 432320975
        #token нашего бота
        token = '7189949223:AAFbFkFXxHtw8oq3C2JbYHFqgI5eqccInnM'


        #параметры подключения к clickhouse
        connection = {
                        'host': 'https://clickhouse.lab.karpov.courses',
                        'password': 'dpo_python_2020',
                        'user': 'student',
                        'database': 'simulator_20240620'
                    }
        #импорт данных из clickhouse
        q = '''
            SELECT DAU,
                   likes,
                   views,
                   likes/views as CTR
            FROM
                (SELECT 
                    sum(distinct(user_id)) as DAU,
                    sum(action = 'like') as likes,
                    sum(action = 'view') as views

                FROM simulator_20240620.feed_actions
                WHERE toDate(time) = yesterday()
                )
            '''

        df = pandahouse.read_clickhouse(q, connection=connection)
        #расчет метрик
        DAU = df.DAU.iloc[0]
        likes = df.likes.iloc[0]
        views = df.views.iloc[0]
        CTR = df.CTR.iloc[0]
        #пишем текст для рассылки
        msg = f"Добрый день! \nОсновные метрики за вчера: \nDAU = {DAU} чел., \nlikes = {likes},  \nviews = {views}, \nCTR = {CTR}."
        #задаем параметры
        params = {'chat_id': chat_id, 'text': msg}

        base_url = f'https://api.telegram.org/bot7189949223:AAFbFkFXxHtw8oq3C2JbYHFqgI5eqccInnM/'
        url = base_url + 'sendMessage?' + urlencode(params)
        #отправляем в чат
        resp = requests.get(url)
        #импорт данных из clickhouse
        DAU = '''
            SELECT 
                toDate(time) as date,
                sum(distinct(user_id)) as DAU
            FROM simulator_20240620.feed_actions
            WHERE toDate(time) >= today() - INTERVAL 6 DAY
            group by date

            '''

        DAU_df = pandahouse.read_clickhouse(DAU, connection=connection)
        #строим график DAU за предыдущую неделю и сохраняем в BytesIO
        sns.lineplot(data=DAU_df, x='date', y='DAU')
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = 'DAU.png'
        plt.close()

        url = base_url + 'sendPhoto?' + urlencode(params)

        # Отправка изображения из BytesIO
        files = {'photo': plot_object}  
        response = requests.post(url, files=files)

    report()

In [57]:
lelkova_report_DAG = lelkova_report_DAG()

После загрузки данного файла в формате .py через репозиторий в Airflow скрипт будет отрабатываться ежедневно в 11.00. Телеграм бот будет направлять рассылку с основными метриками и графиком в указанный чат.